# Exporting model from Chainer to ONNX

In thi tutorial, we describe how to use ONNX-Chainer to convert a model defined in Chainer into the ONNX format.

ONNX export is provided as a separte package [onnx-chainer](https://github.com/chainer/onnx-chainer). You can install it via pip like this:

```
pip install onnx-chainer
```

`onnx_chainer` provides `export` function that takes a Chainer model and its expected arguments given to `__call__` method of the model. It executes a forward pass once with the given model object and arguments to construct a computational graph. Because Chainer is the first deep learning framework that proposed Define-by-Run approach, the computational graph for backward computation is constructed on-the-fly. `onnx-chainer` is a trace-based exporter, so it needs to run a model once before converting the structure into ONNX.

`onnx_chainer.export()` function have some other options for example `filename` which is to save the converted model to a disk.

## Limitations

The [onnx-chainer](https://github.com/chainer/onnx-chainer) currently does not support exporting dynamic models which change their behavior depending on input data, because ONNX format currently cannot represent such dynamic behavior.

Additionally, some ONNX operators, for example, `Reshape`, keeps the explicit batch size in the shape of dummy input data, so that the exported ONNX model will be runnable only with the same batch size. You may want to run the model with different batch size at inference time, but it might fail due to this limitation. To modify the batch size in the `Reshape` operator of ONNX, you need to modify the value by hand after exporting.

In [1]:
import onnx_chainer
help(onnx_chainer.export)

Help on function export in module onnx_chainer.export:

export(model, args, filename=None, export_params=True, graph_name='Graph', save_text=False)
    Export function for chainer.Chain in ONNX format.
    
    This function performs a forward computation of the given
    :class:`~chainer.Chain`, ``model``, by passing the given argments ``args``
    directly. It means, the output :class:`~chainer.Variable` object ``y`` to
    make the computational graph will be created by:
    
    y = model(*args)
    
    Args:
        model (~chainer.Chain): The model object you want to export in ONNX
            format. It should have :meth:`__call__` method because the second
            argment ``args`` is directly given to the model by the ``[]``
            accessor.
        args (list or dict): The argments which are given to the model
            directly.
        filename (str or file-like object): The filename used for saving the
            resulting ONNX model. If None, nothing is saved 

## Export VGG16 into ONNX

Chainer provides VGG16 implementation with pre-trained weights, so let's see how the network that is a `chainer.Chain` object can be exported into ONNX.

In [2]:
import numpy as np
import chainer
import chainer.links as L
import onnx_chainer

model = L.VGG16Layers()

# Pseudo input
x = np.zeros((1, 3, 224, 224), dtype=np.float32)

# Don't forget to set train flag off!
chainer.config.train = False

onnx_chainer.export(model, x, filename='output/VGG16.onnx')

**That's all.**

Now you can find the exported ONNX binary named `VGG16.onnx` under the `output` dir.

## Inspecting the exported model

You can use ONNX tooling to check the validity of the exported ONNX model and inspect the details.

In [4]:
import onnx

# Load the ONNX model
model = onnx.load("output/VGG16.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph Graph (
  %4704767504[FLOAT, 1x3x224x224]
) initializers (
  %/conv5_1/b[FLOAT, 512]
  %/conv5_1/W[FLOAT, 512x512x3x3]
  %/conv5_3/b[FLOAT, 512]
  %/conv5_3/W[FLOAT, 512x512x3x3]
  %/conv3_2/b[FLOAT, 256]
  %/conv3_2/W[FLOAT, 256x256x3x3]
  %/fc6/b[FLOAT, 4096]
  %/fc6/W[FLOAT, 4096x25088]
  %/fc8/b[FLOAT, 1000]
  %/fc8/W[FLOAT, 1000x4096]
  %/conv1_2/b[FLOAT, 64]
  %/conv1_2/W[FLOAT, 64x64x3x3]
  %/conv2_1/b[FLOAT, 128]
  %/conv2_1/W[FLOAT, 128x64x3x3]
  %/conv4_1/b[FLOAT, 512]
  %/conv4_1/W[FLOAT, 512x256x3x3]
  %/conv2_2/b[FLOAT, 128]
  %/conv2_2/W[FLOAT, 128x128x3x3]
  %/conv3_3/b[FLOAT, 256]
  %/conv3_3/W[FLOAT, 256x256x3x3]
  %/conv1_1/b[FLOAT, 64]
  %/conv1_1/W[FLOAT, 64x3x3x3]
  %/conv3_1/b[FLOAT, 256]
  %/conv3_1/W[FLOAT, 256x128x3x3]
  %/conv4_3/b[FLOAT, 512]
  %/conv4_3/W[FLOAT, 512x512x3x3]
  %/conv5_2/b[FLOAT, 512]
  %/conv5_2/W[FLOAT, 512x512x3x3]
  %/fc7/b[FLOAT, 4096]
  %/fc7/W[FLOAT, 4096x4096]
  %/conv4_2/b[FLOAT, 512]
  %/conv4_2/W[FLOAT, 512x512x3x3]
) {
  %47

You can see all the parameters and layers in the VGG16 model here. The actual values those parameters have are stored in `model.graph.initializers`.